<a href="https://colab.research.google.com/github/jackschedel/AutoCalibr/blob/main/AutoCalibr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset importing

In [ ]:
#@title Define imports and constants
from google.colab import drive
import time

drive.mount('/content/drive', force_remount=True)

# dataset:
# fbx files (main dataset source)
FBX = '/content/drive/MyDrive/AutoCalibr/dataset/fbx/'
# ply (pre-converted to fix artifacts)
PLY_TO_IMPORT = '/content/drive/MyDrive/AutoCalibr/dataset/ply/'

# intermediate folders
INTERMEDIATES = '/content/intermediates/'
CONVERTED_PLY = '/content/intermediates/converted_ply/'
NORMALIZED_PLY = '/content/intermediates/normalized_ply/'
# NORMALIZED_PLY = '/content/drive/MyDrive/AutoCalibr/normalized_ply/'


# sometimes used for debug outputting into non-cluttered directory
DIR = '/content/'

# these will be automatically determined, do not change them (defined here to allow out-of-order exports)
global_stretch_scale_x = 1
global_stretch_scale_y = 1
global_stretch_scale_z = 1

!rm -r sample_data/ 2>/dev/null
!mkdir {INTERMEDIATES} 2>/dev/null

Mounted at /content/drive


In [ ]:
#@title Convert FBX to PLY {vertical-output: true}

#!pip install bpy
!rm -r {CONVERTED_PLY} 2>/dev/null
!mkdir {CONVERTED_PLY}

import os

try:
  import bpy
except ImportError:
  !pip install bpy
  import bpy

bpy.ops.wm.read_factory_settings()

# needs to be rotated counterclockwise (left) 90 degrees - could be mostly automated (noted below)
needs_extra_rotation = ["Ace Of Spades", "Cantata-57", "Chroma Rush", "Cloudstrike", "Dead Mans Tale", "Duality", "False Promises", "Fugue 55", "Hawkmoon", "Jack Queen King 3", "Mindbenders Ambition", "No Time To Explain", "Ruinous Effigy", "Seven Seraph Carbine", "Seventh Seraph CQC-12", "Seventh Seraph Officer Revolver", "Seventh Seraph SAW", "Seventh Seraph SI-2", "Seventh Seraph VY-7", "The Fourth Horseman", "Trustee", "Witherhoard"]

for f in os.listdir(FBX):
  if f.endswith('.fbx'):
    # Isolate the name of the .fbx file (without extension)
    name_no_ext = os.path.splitext(os.path.basename(f))[0]

    print(f"Object: {name_no_ext}\n")

    # Delete all mesh objects to avoid exporting multiple models into the same file
    bpy.ops.object.select_all(action='DESELECT')
    bpy.ops.object.select_by_type(type='MESH')
    bpy.ops.object.delete()

    # Load in FBX file
    bpy.ops.import_scene.fbx(filepath=os.path.join(FBX, f))

    # Select the object
    obj_object = bpy.context.selected_objects[0]
    bpy.context.view_layer.objects.active = obj_object

    if name_no_ext in needs_extra_rotation:
      forwards_dir = 'Z'
      needs_extra_rotation.remove(name_no_ext)
    else:
      forwards_dir = '-X'

    # Export object to PLY
    bpy.ops.export_mesh.ply(filepath=os.path.join(CONVERTED_PLY, f.replace('.fbx', '.ply')), use_ascii=True, use_mesh_modifiers=True, use_normals=False, use_uv_coords=False, use_colors=False, axis_forward=forwards_dir, axis_up='Y')

    print('-' * 50)

# Ensure that any models that were supposed to receive extra rotation were hit
if len(needs_extra_rotation) > 0:
  print(f"\nThe following manually-specified models were not hit (check for typos): {needs_extra_rotation}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.5 MB/s eta 0:00:00
Object: Coldheart

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Coldheart.ply' in 0.353
--------------------------------------------------
Object: Vigilance Wing

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Vigilance Wing.ply' in 0.339
--------------------------------------------------
Object: Riskrunner

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Riskrunner.ply' in 0.297
--------------------------------------------------
Object: The Last Word

FBX version: 7400
Export completed '/content/intermediates/converted_ply/The Last Word.ply' in 0.086
--------------------------------------------------
Object: Lord of Wolves

FBX version: 7400
Export completed '/content/intermediates/converted_ply/Lord of Wolves.ply' in 0.220
--------------------------

# Dataset processing

In [ ]:
# @title Define PlyObject Class
import random
from random import choice

class Vertex:
  def __init__(self, x, y, z):
    self.x = x
    self.y = y
    self.z = z


  def scale(self, scale_x, scale_y, scale_z):
    self.x *= scale_x
    self.y *= scale_y
    self.z *= scale_z


  def translate(self, offset_x, offset_y, offset_z):
    self.x += offset_x
    self.y += offset_y
    self.z += offset_z


  def __hash__(self):
    return hash((self.x, self.y, self.z))


  def __eq__(self, other):
    if isinstance(other, Vertex):
      return self.x == other.x and self.y == other.y and self.z == other.z
    return False


class Face:
  def __init__(self, vertices):
    # vertices is a list of indexes (of the object's vertices list) of the connected vertices that form the face
    self.vertices = vertices


  def __lt__(self, other):
    for val1, val2 in zip(self.vertices, other.vertices):
      if val1 < val2:
        return True
      elif val1 > val2:
        return False
    return len(self.vertices) < len(other.vertices)


  def __hash__(self):
    return hash(tuple(self.vertices))


  def __eq__(self, other):
    if isinstance(other, Face):
      return (sorted(self.vertices) == sorted(other.vertices))
    return False


class PlyObject:
  def __init__(self, name, vertices, faces):
    self.name = name
    self.vertices = vertices
    self.faces = faces


  def save_file(self, filename):
    with open(filename, "w") as file:
      file.write("ply\n")
      file.write("format ascii 1.0\n")
      file.write("comment Created by PlyObject class\n")
      file.write(f"element vertex {len(self.vertices)}\n")
      file.write("property float x\n")
      file.write("property float y\n")
      file.write("property float z\n")
      file.write(f"element face {len(self.faces)}\n")
      file.write("property list uchar uint vertex_indices\n")
      file.write("end_header\n")

      for vertex in self.vertices:
        file.write(f"{vertex.x} {vertex.y} {vertex.z}\n")

      for face in self.faces:
        formatted_vertices = ' '.join(str(v) for v in face.vertices)
        number_of_vertices = len(face.vertices)
        file.write(f"{number_of_vertices} {formatted_vertices}\n")


  def scale(self, scale_x, scale_y, scale_z):
    for vertex in self.vertices:
      vertex.scale(scale_x, scale_y, scale_z)


  def translate(self, offset_x, offset_y, offset_z):
    for vertex in self.vertices:
      vertex.translate(offset_x, offset_y, offset_z)


  def calculate_volume(self):
    volume = 0
    for face in self.faces:
      v0 = self.vertices[face.vertices[0]]
      v1 = self.vertices[face.vertices[1]]
      v2 = self.vertices[face.vertices[2]]
      volume += (-v0.x*v1.y*v2.z + v1.x*v0.y*v2.z + v0.x*v2.y*v1.z - v2.x*v0.y*v1.z + v2.x*v1.y*v0.z - v1.x*v2.y*v0.z)
    return abs(volume) / 6.0


  def remove_overlapping(self):
    vert_dict = {}
    convert_dict = {}

    new_vertices = []
    # iterate over existing vertices to identify and save unique ones
    for idx, vertex in enumerate(self.vertices):
      if vertex in vert_dict:
        convert_dict[idx] = vert_dict[vertex]
      else:
        # assign a unique index to each vertex
        vert_dict[vertex] = len(new_vertices)
        convert_dict[idx] = len(new_vertices)
        new_vertices.append(vertex)

    # replace original vertices with new, duplicate-free list
    self.vertices = new_vertices

    # apply convert_dict to update face vertices to match new unique indexing
    for face in self.faces:
      face.vertices = [convert_dict[vertex] for vertex in face.vertices]

    # convert faces to set to remove any potential duplicate faces
    self.faces = set(self.faces)
    self.faces = list(set(self.faces))


  # not great for scale since re-sorting is needed, use a batch version after dataset processing
  def add_random_duplicate_vertices(self, count_to_add):
    # note: will need to re-sort tri data if already sorted
    for _ in range(count_to_add):
      to_duplicate = choice(self.vertices)
      new_vertex = Vertex(to_duplicate.x, to_duplicate.y, to_duplicate.z)
      self.vertices.append(new_vertex)


  # not great for scale since re-sorting is needed, use a batch version after dataset processing
  def add_random_duplicate_faces(self, count_to_add):
    # note: will need to re-sort tri data if already sorted
    for _ in range(count_to_add):
      to_duplicate = choice(self.faces)
      new_face = Face(to_duplicate.vertices)
      self.faces.append(new_face)


  def subdivide_face(self, face_index):
    # note: loses sorting
    if face_index >= len(self.faces):
        return

    # Get the face to be subdivided
    face = self.faces[face_index]

    # Find vertices to split between and create new vertex in between
    vertex_index_1, vertex_index_2 = face.vertices[:2]
    vertex_1, vertex_2 = self.vertices[vertex_index_1], self.vertices[vertex_index_2]

    new_vertex = Vertex((vertex_1.x + vertex_2.x) / 2, (vertex_1.y + vertex_2.y) / 2, (vertex_1.z + vertex_2.z) / 2)

    # Add new_vertex to the vertices list and store its index
    self.vertices.append(new_vertex)
    new_vertex_index = len(self.vertices) - 1

    # Create two new faces
    new_faces = [Face([vertex_index_1, new_vertex_index, face.vertices[2]]),
                Face([vertex_index_2, new_vertex_index, face.vertices[2]])]

    # Replace the old face with the new faces
    self.faces[face_index] = new_faces[0]
    self.faces.append(new_faces[1])


  def subdivide_faces_as_padding(self, max_faces, max_vertices):
    # note: loses sorted status
    while len(self.faces) < max_faces and len(self.vertices) < max_vertices:
      face_index = random.randint(0, len(self.faces) - 1)
      obj.subdivide_face(face_index)


  def categorize_faces(self):
    face_dict = {}

    for face in obj.faces:
      length = len(face.vertices)

      if length in face_dict:
        face_dict[length] = face_dict[length] + 1
      else:
        face_dict[length] = 1

    return face_dict


  def get_value_extrema(self):
    min_val = float('inf')
    max_val = float('-inf')

    for v in self.vertices:
      min_val = min(min_val, v.x, v.y, v.z)
      max_val = max(max_val, v.x, v.y, v.z)

    return {'min': min_val, 'max': max_val}


  def get_max_values(self):
    max_values = {'x': None, 'y': None, 'z': None}

    for vertex in self.vertices:
      if max_values['x'] is None or vertex.x > max_values['x']:
        max_values['x'] = vertex.x
      if max_values['y'] is None or vertex.y > max_values['y']:
        max_values['y'] = vertex.y
      if max_values['z'] is None or vertex.z > max_values['z']:
        max_values['z'] = vertex.z

    return max_values


  def get_min_values(self):
    min_values = {'x': None, 'y': None, 'z': None}

    for vertex in self.vertices:
      if min_values['x'] is None or vertex.x < min_values['x']:
        min_values['x'] = vertex.x
      if min_values['y'] is None or vertex.y < min_values['y']:
        min_values['y'] = vertex.y
      if min_values['z'] is None or vertex.z < min_values['z']:
        min_values['z'] = vertex.z

    return min_values


  def center_object(self):
    max_vals = self.get_max_values()
    min_vals = self.get_min_values()

    offset_vals = {'x': 0, 'y': 0, 'z': 0}

    for dim in offset_vals:
      center = (max_vals[dim] + min_vals[dim]) / 2
      offset_vals[dim] = -center

    self.translate(offset_vals['x'], offset_vals['y'], offset_vals['z'])


  def normalize_scale(self):
    x_coordinates = [vertex.x for vertex in self.vertices]
    y_coordinates = [vertex.y for vertex in self.vertices]
    z_coordinates = [vertex.z for vertex in self.vertices]

    max_distance = max(x_coordinates + y_coordinates + z_coordinates)
    min_distance = min(x_coordinates + y_coordinates + z_coordinates)
    normalization_range = max_distance - min_distance

    if normalization_range == 0:
      raise ValueError("Normalization range cannot be zero")

    for vertex in self.vertices:
      vertex.x = 2 * (vertex.x - min_distance) / normalization_range - 1
      vertex.y = 2 * (vertex.y - min_distance) / normalization_range - 1
      vertex.z = 2 * (vertex.z - min_distance) / normalization_range - 1


  def squares_to_tris(self):
    new_faces = []
    for face in self.faces:
      if len(face.vertices) == 4:
        new_faces.append(Face([face.vertices[0], face.vertices[1], face.vertices[2]]))
        new_faces.append(Face([face.vertices[0], face.vertices[2], face.vertices[3]]))
      else:
        new_faces.append(face)

    self.faces = new_faces


  def delete_plane(self):
    # Note: only call if the object has a plane artifact!
    max_x_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    min_x_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].x)
    max_z_index = max(range(len(self.vertices)), key = lambda index: self.vertices[index].z)
    min_z_index = min(range(len(self.vertices)), key = lambda index: self.vertices[index].z)

    indices_to_remove = set([max_x_index, min_x_index, max_z_index, min_z_index])

    for f, face in enumerate(self.faces):
      if set(face.vertices).intersection(indices_to_remove) == indices_to_remove:
        del self.faces[f]
        break
    else:
      raise ValueError("Face with all vertices not found")

    for index in sorted(indices_to_remove, reverse=True):
      del self.vertices[index]

    for face in self.faces:
      face.vertices = [idx if idx not in indices_to_remove else -1 for idx in face.vertices]

    self.remove_disconnected_vertices()


  def remove_disconnected_vertices(self):
    connected_vertices = set()
    for face in self.faces:
      connected_vertices |= set(face.vertices)

    shift_indices = []
    new_vertices = []
    for idx, vertex in enumerate(self.vertices):
      if idx in connected_vertices:
        new_vertices.append(vertex)
        shift_indices.append(len(new_vertices) - 1)
      else:
        shift_indices.append(None)

    self.vertices = new_vertices

    for face in self.faces:
      face.vertices = [shift_indices[vertex] if shift_indices[vertex] is not None else None for vertex in face.vertices]
      face.vertices = [vertex for vertex in face.vertices if vertex is not None]

    self.faces = [face for face in self.faces if face.vertices]


  def stretch_to_max(self):
    # note: update scale globals before calling
    self.scale(global_stretch_scale_x, global_stretch_scale_y, global_stretch_scale_z)


  def revert_stretch(self):
    self.scale(1/global_stretch_scale_x, 1/global_stretch_scale_y, 1/global_stretch_scale_z)


  def dimension_range(self):
    xmin = xmax = obj.vertices[0].x
    ymin = ymax = obj.vertices[0].y
    zmin = zmax = obj.vertices[0].z

    for vertex in obj.vertices:
      xmin = min(xmin, vertex.x)
      xmax = max(xmax, vertex.x)
      ymin = min(ymin, vertex.y)
      ymax = max(ymax, vertex.y)
      zmin = min(zmin, vertex.z)
      zmax = max(zmax, vertex.z)

    xrang = xmax - xmin
    yrang = ymax - ymin
    zrang = zmax - zmin

    return {'x':xrang, 'y':yrang, 'z':zrang}


  def sort_tri_data(self):
    index_map = {}

    # sort the vertices by x first, then y, then z
    sorted_vertices = sorted(
      enumerate(self.vertices),
      key=lambda pair: (pair[1].x, pair[1].y, pair[1].z)
    )
    self.vertices = [pair[1] for pair in sorted_vertices]

    # record the new indices of the vertices in the map
    for i, pair in enumerate(sorted_vertices):
      old_index, _ = pair
      index_map[old_index] = i

    # convert old face lists to new face lists using the index map
    new_faces = []
    for face in self.faces:
      new_face_vertices = [index_map[i] for i in face.vertices]
      new_faces.append(Face(new_face_vertices))

    # replace old face list with new face list, which we first sort
    new_faces.sort()

    self.faces = new_faces


  @classmethod
  def from_file(cls, filepath):
    with open(filepath, 'r') as f:
      if next(f).strip() != "ply":
        raise ValueError("The file being read is not a PLY file.")

      for _ in range(2):
        next(f)

      n_vertices = int(next(f).split()[-1])

      for _ in range(3):
        next(f)

      n_faces = int(next(f).split()[-1])

      for _ in range(2):
        next(f)

      vertices = []
      for _ in range(n_vertices):
        x, y, z = map(float, next(f).split())
        vertices.append(Vertex(x, y, z))

      faces = []
      for _ in range(n_faces):
        face_vertices = list(map(int, next(f).split()[1:]))
        faces.append(Face(face_vertices))

    name = os.path.splitext(os.path.basename(filepath))[0]

    return cls(name, vertices, faces)

In [ ]:
#@title Process PLY files as PlyObject {vertical-output: true}

import os

# Toggle between using full dataset and just preconverted subset
use_fbx_dataset = False
use_ply_dataset = True
use_single_test_file = False

ply_objs = []

if use_fbx_dataset:
  for f in os.listdir(CONVERTED_PLY):
    if f.endswith('.ply'):
      start_time = time.time()
      obj = PlyObject.from_file(filepath=os.path.join(CONVERTED_PLY, f))
      ply_objs.append(obj)
      elapsed_time = time.time() - start_time

      print(f"Object: {obj.name}\n")
      print(f'Processed auto-converted PLY into PlyObject (took {elapsed_time:.2f} s)')
      print(f'\nVertice count: {len(obj.vertices)}')
      print(f'Face count: {len(obj.faces)}')
      print('-' * 50)


# Import pre-converted .ply files
if use_ply_dataset:
  for f in os.listdir(PLY_TO_IMPORT):
    if f.endswith('.ply'):
      start_time = time.time()
      obj = PlyObject.from_file(filepath=os.path.join(PLY_TO_IMPORT, f))
      ply_objs.append(obj)
      elapsed_time = time.time() - start_time

      print(f"Object: {obj.name}\n")
      print(f'Processed pre-converted PLY into PlyObject (took {elapsed_time:.2f} s)')
      print(f'\nVertice count: {len(obj.vertices)}')
      print(f'Face count: {len(obj.faces)}')
      print('-' * 50)

if (not use_ply_dataset) and use_single_test_file:
  f = PLY_TO_IMPORT+'Blind Perdition.ply'
  start_time = time.time()
  obj = PlyObject.from_file(filepath=os.path.join(PLY_TO_IMPORT, f))
  ply_objs.append(obj)
  elapsed_time = time.time() - start_time

  print(f"Object: {obj.name}\n")
  print(f'Processed pre-converted PLY into PlyObject (took {elapsed_time:.2f} s)')
  print(f'\nVertice count: {len(obj.vertices)}')
  print(f'Face count: {len(obj.faces)}')
  print('-' * 50)

Object: Line in the Sand

Processed pre-converted PLY into PlyObject (took 0.32 s)

Vertice count: 24644
Face count: 29692
--------------------------------------------------
Object: Rat King

Processed pre-converted PLY into PlyObject (took 0.40 s)

Vertice count: 12219
Face count: 24308
--------------------------------------------------
Object: Outbreak Perfected

Processed pre-converted PLY into PlyObject (took 0.70 s)

Vertice count: 86406
Face count: 28802
--------------------------------------------------
Object: Prometheus Lens

Processed pre-converted PLY into PlyObject (took 0.44 s)

Vertice count: 13902
Face count: 24149
--------------------------------------------------
Object: Blind Perdition

Processed pre-converted PLY into PlyObject (took 0.07 s)

Vertice count: 13044
Face count: 14070
--------------------------------------------------


In [ ]:
#@title Remove plane artifacts (dataset specific cleaning) {vertical-output: true}

# has a giant rectangular plane originally used as a background, will need to be filtered out
has_plane_artifact = ['Abbadon', 'Blind Perdition', 'Ex Machina', 'Komodo-4FR', 'Nova Mortis', 'Trespasser', 'Vestian Dynasty', 'Vouchsafe', 'Hereafter']

# Remove plane artifact from manually specified objects

# Could also iterate over all objects and use exceptions,
# but would need to give delete_plane function stricter pre-deletion checking
for obj in ply_objs:
  if obj.name in has_plane_artifact:
    has_plane_artifact.remove(obj.name)

    obj_range = obj.dimension_range()

    print(f"Object: {obj.name}")
    print("\nDimension Range:")
    print('X:', obj_range['x'])
    print('Y:', obj_range['y'])
    print('Z:', obj_range['z'])

    try:
      start_time = time.time()

      obj.delete_plane()

      elapsed_time = time.time() - start_time

      obj_range = obj.dimension_range()

      print(f"\nPlane detected and deleted (took {elapsed_time:.2f} s)")
      print(f"\nDimension Range (plane deleted in {elapsed_time:.2f} s):")
      print('X:', obj_range['x'])
      print('Y:', obj_range['y'])
      print('Z:', obj_range['z'])
    except ValueError:
      print(f"\nNo plane found!")

    print('-' * 50)

if len(has_plane_artifact) > 0:
  print(f"\nThe following manually-specified models were not hit (check for typos): {has_plane_artifact}")

Object: Blind Perdition

Dimension Range:
X: 1.831408
Y: 0.47424900000000003
Z: 3.43575

Plane detected and deleted (took 0.07 s)

Dimension Range (plane deleted in 0.07 s):
X: 0.10221
Y: 0.46234
Z: 1.319475
--------------------------------------------------

The following manually-specified models were not hit (check for typos): ['Abbadon', 'Ex Machina', 'Komodo-4FR', 'Nova Mortis', 'Trespasser', 'Vestian Dynasty', 'Vouchsafe', 'Hereafter']


In [ ]:
#@title Categorize face data to check for any bad n-gons

face_lengths = {}

for obj in ply_objs:
  temp_face_lengths = obj.categorize_faces()

  for length, count in temp_face_lengths.items():
    if length in face_lengths:
      face_lengths[length] += count
    else:
      face_lengths[length] = count

for length, count in face_lengths.items():
  print(f'Faces with {length} vertices: {count} instances')

for length, count in face_lengths.items():
  if length < 3 or length > 4:
    raise Exception(f'\nFace of unsupported size {length}!')

Faces with 3 vertices: 119728 instances
Faces with 4 vertices: 1292 instances


In [ ]:
#@title Convert any objects with square faces to tris {vertical-output: true}

# this could probably done using bpy before exporting as a PLY but this allows this to be done for any PlyObject

face_lengths = {}
square_obj_count = 0
for obj in ply_objs:

  face_data = obj.categorize_faces()

  if 4 in face_data:
    square_obj_count = square_obj_count + 1

    print(f"Object: {obj.name}\n")
    if 3 in face_data:
      print(f"Tris: {face_data[3]}")
    else:
      print(f"Squares: 0")
    print(f"Squares: {face_data[4]}")

    start_time = time.time()

    obj.squares_to_tris()

    elapsed_time = time.time() - start_time

    print(f'\nConverted square faces to tris (took {elapsed_time:.2f} s)\n')
    face_data = obj.categorize_faces()
    print(f"Tris: {face_data[3]}")
    if 4 in face_data:
      print(f"Squares: {face_data[4]}")
    else:
      print(f"Squares: 0")

    print('-' * 50)

if square_obj_count == 0:
  print(f"No objects containing squares found. All objects contain only tris.")

Object: Prometheus Lens

Tris: 22857
Squares: 1292

Converted square faces to tris (took 0.02 s)

Tris: 25441
Squares: 0
--------------------------------------------------


In [ ]:
#@title Merge overlapping vertices and faces {vertical-output: true}

for obj in ply_objs:
    start_time = time.time()

    obj.remove_overlapping()

    elapsed_time = time.time() - start_time

    print(f"Object: {obj.name}\n")
    print(f'Merged any overlaping vertices and faces (took {elapsed_time:.2f} s)')
    print('-' * 50)

Object: Line in the Sand

Merged any overlaping vertices and faces (took 0.27 s)
--------------------------------------------------
Object: Rat King

Merged any overlaping vertices and faces (took 0.16 s)
--------------------------------------------------
Object: Outbreak Perfected

Merged any overlaping vertices and faces (took 0.30 s)
--------------------------------------------------
Object: Prometheus Lens

Merged any overlaping vertices and faces (took 0.04 s)
--------------------------------------------------
Object: Blind Perdition

Merged any overlaping vertices and faces (took 0.04 s)
--------------------------------------------------


In [ ]:
#@title Center the objects to the origin {vertical-output: true}

for obj in ply_objs:
    start_time = time.time()

    obj.center_object()

    elapsed_time = time.time() - start_time

    print(f"Object: {obj.name}\n")

    print(f'Centered to origin (took {elapsed_time:.2f} s)')

    print('-' * 50)

Object: Line in the Sand

Centered to origin (took 0.04 s)
--------------------------------------------------
Object: Rat King

Centered to origin (took 0.02 s)
--------------------------------------------------
Object: Outbreak Perfected

Centered to origin (took 0.02 s)
--------------------------------------------------
Object: Prometheus Lens

Centered to origin (took 0.02 s)
--------------------------------------------------
Object: Blind Perdition

Centered to origin (took 0.02 s)
--------------------------------------------------


In [ ]:
#@title Normalize ondividual object scale to perfectly fit boundaries {vertical-output: true}

for obj in ply_objs:

  extrema = obj.get_value_extrema()

  print(f"Object: {obj.name}")
  print("\nExtrema in any Dimension:")
  print(f"Minimum: {extrema['min']}")
  print(f"Maximum: {extrema['max']}")


  start_time = time.time()

  obj.normalize_scale()

  elapsed_time = time.time() - start_time

  print(f'\nNormalised object scale to boundaries (took {elapsed_time:.2f} s)')

  extrema = obj.get_value_extrema()

  print("\nExtrema in any Dimension:")
  print(f"Minimum: {extrema['min']}")
  print(f"Maximum: {extrema['max']}")
  print('-' * 50)

Object: Line in the Sand

Extrema in any Dimension:
Minimum: -1.0
Maximum: 1.0

Normalised object scale to boundaries (took 0.02 s)

Extrema in any Dimension:
Minimum: -1.0
Maximum: 1.0
--------------------------------------------------
Object: Rat King

Extrema in any Dimension:
Minimum: -0.1960095
Maximum: 0.1960095

Normalised object scale to boundaries (took 0.02 s)

Extrema in any Dimension:
Minimum: -1.0
Maximum: 1.0
--------------------------------------------------
Object: Outbreak Perfected

Extrema in any Dimension:
Minimum: -0.4440425
Maximum: 0.4440425

Normalised object scale to boundaries (took 0.04 s)

Extrema in any Dimension:
Minimum: -1.0
Maximum: 1.0
--------------------------------------------------
Object: Prometheus Lens

Extrema in any Dimension:
Minimum: -1.0
Maximum: 1.0

Normalised object scale to boundaries (took 0.02 s)

Extrema in any Dimension:
Minimum: -1.0
Maximum: 1.0
--------------------------------------------------
Object: Blind Perdition

Extrema in

In [ ]:
#@title Calculate minimum volume of any object

volumes = {}
min_volume = float('inf')
volume_calc_times = {}

for obj in ply_objs:
    start_time = time.time()

    volumes[obj.name] = obj.calculate_volume()

    volume_calc_times[obj.name] = time.time() - start_time

    min_volume = min(min_volume, volumes[obj.name])

print(f'Global minimum volume: {min_volume:.6f}')

Global minimum volume: 0.028479


In [ ]:
#@title Scale each object to match the minimum global volume {vertical-output: true}

# just scaled to boundaries so the objects must be scaled down, not up

import math

for obj in ply_objs:
    print(f"Object: {obj.name}\n")
    print(f'Volume calculated as: {volumes[obj.name]:.6f} (took {volume_calc_times[obj.name]:.2f} s)')
    start_time = time.time()

    to_scale = math.pow(min_volume / volumes[obj.name], 1/3)
    obj.scale(to_scale, to_scale, to_scale)

    elapsed_time = time.time() - start_time
    print(f'\nScaled by {to_scale} (took {elapsed_time:.2f} s)')
    start_time = time.time()

    volume = obj.calculate_volume()

    elapsed_time = time.time() - start_time
    print(f'\nVolume calculated as: {volume:.6f} (took {elapsed_time:.2f} s)')
    print('-' * 50)

Object: Line in the Sand

Volume calculated as: 0.028479 (took 0.16 s)

Scaled by 1.0 (took 0.01 s)

Volume calculated as: 0.028479 (took 0.12 s)
--------------------------------------------------
Object: Rat King

Volume calculated as: 0.272199 (took 0.12 s)

Scaled by 0.47121092908297674 (took 0.00 s)

Volume calculated as: 0.028479 (took 0.13 s)
--------------------------------------------------
Object: Outbreak Perfected

Volume calculated as: 0.065950 (took 0.11 s)

Scaled by 0.7558567953805982 (took 0.02 s)

Volume calculated as: 0.028479 (took 0.23 s)
--------------------------------------------------
Object: Prometheus Lens

Volume calculated as: 0.050188 (took 0.11 s)

Scaled by 0.8278999959990196 (took 0.01 s)

Volume calculated as: 0.028479 (took 0.35 s)
--------------------------------------------------
Object: Blind Perdition

Volume calculated as: 0.044829 (took 0.05 s)

Scaled by 0.8596580597884671 (took 0.01 s)

Volume calculated as: 0.028479 (took 0.24 s)
-------------

In [ ]:
#@title Offset objects to pin Z maximum to upper boundary to localize similar features (dataset specific scope reduction) {vertical-output: true}

# pinning all objects against back wall

for obj in ply_objs:
  start_time = time.time()

  max_z = obj.get_max_values()['z']
  z_offset = 1 - max_z
  obj.translate(0, 0, z_offset)
  new_max_z = obj.get_max_values()['z']
  elapsed_time = time.time() - start_time



  print(f"Object: {obj.name}")
  print(f"\nMaximum Z: {max_z}")
  print(f'\nOffset object to upper boundary (took {elapsed_time:.2f} s)')
  print(f"\nMaximum Z: {new_max_z}")
  print('-' * 50)

Object: Line in the Sand

Maximum Z: 1.0

Offset object to upper boundary (took 0.07 s)

Maximum Z: 1.0
--------------------------------------------------
Object: Rat King

Maximum Z: 0.47121092908297674

Offset object to upper boundary (took 0.03 s)

Maximum Z: 1.0
--------------------------------------------------
Object: Outbreak Perfected

Maximum Z: 0.7558567953805982

Offset object to upper boundary (took 0.03 s)

Maximum Z: 1.0
--------------------------------------------------
Object: Prometheus Lens

Maximum Z: 0.8278999959990196

Offset object to upper boundary (took 0.03 s)

Maximum Z: 1.0
--------------------------------------------------
Object: Blind Perdition

Maximum Z: 0.8596580597884671

Offset object to upper boundary (took 0.04 s)

Maximum Z: 1.0
--------------------------------------------------


In [ ]:
#@title Analyze dimension ranges to pseudo-verify object orientation

# could do anywhere before doing global stretch
# choosing to do it after all other processing

# used for checking if I missed any rotation overrides
count_smallest = {'x': 0, 'y': 0, 'z': 0}
count_middle = {'x': 0, 'y': 0, 'z': 0}
count_largest = {'x': 0, 'y': 0, 'z': 0}

for obj in ply_objs:

  obj_range = obj.dimension_range()

  # count the podium placings of ranges for every dimension
  sorted_keys = sorted(obj_range, key=obj_range.get)
  smallest_key = sorted_keys[0]
  middle_key = sorted_keys[1]
  largest_key = sorted_keys[2]

  count_smallest[smallest_key] += 1
  count_middle[middle_key] += 1
  count_largest[largest_key] += 1

print("Times with largest dimensional range:")
print(f"X: {count_largest['x']}")
print(f"Y: {count_largest['y']}")
print(f"Z: {count_largest['z']}")

print("\nTimes with middle dimensional range:")
print(f"X: {count_middle['x']}")
print(f"Y: {count_middle['y']}")
print(f"Z: {count_middle['z']}")

print("\nTimes with smallest dimensional range:")
print(f"X: {count_smallest['x']}")
print(f"Y: {count_smallest['y']}")
print(f"Z: {count_smallest['z']}")

Times with largest dimensional range:
X: 0
Y: 0
Z: 5

Times with middle dimensional range:
X: 0
Y: 5
Z: 0

Times with smallest dimensional range:
X: 5
Y: 0
Z: 0


In [ ]:
#@title Calculate global stretch values given global dimensional extrema

min_x = min_y = min_z = float('inf')
max_x = max_y = max_z = float('-inf')

for obj in ply_objs:
  for vertex in obj.vertices:
    min_x = min(min_x, vertex.x)
    min_y = min(min_y, vertex.y)
    min_z = min(min_z, vertex.z)
    max_x = max(max_x, vertex.x)
    max_y = max(max_y, vertex.y)
    max_z = max(max_z, vertex.z)

scale_x, scale_y, scale_z = 2/(max_x - min_x), 2/(max_y - min_y), 2/(max_z - min_z)

print(f"Dimensional minima:")
print(f"X: {min_x}")
print(f"Y: {min_y}")
print(f"Z: {min_z}")

print(f"\nDimensional maxima:")
print(f"X: {max_x}")
print(f"Y: {max_y}")
print(f"Z: {max_z}\n")

print('-' * 50)

print(f"\nDerived Stretch Value:")
print(f"X: {scale_x}")
print(f"Y: {scale_y}")
print(f"Z: {scale_z}")

Dimensional minima:
X: -0.12194939584895402
Y: -0.33554149999999994
Z: -1.0

Dimensional maxima:
X: 0.12194939584895394
Y: 0.33554149999999994
Z: 1.0

--------------------------------------------------

Derived Stretch Value:
X: 8.200122624950072
Y: 2.9802572856114673
Z: 1.0


In [ ]:
#@title Stretch non-bounded dimensions using constant values to tighten scope {vertical-output: true}

global_stretch_scale_x = scale_x
global_stretch_scale_y = scale_y
global_stretch_scale_z = scale_z

for obj in ply_objs:
  obj_range = obj.dimension_range()

  print(f"Object: {obj.name}")
  print("\nDimension Range:")
  print('X:', obj_range['x'])
  print('Y:', obj_range['y'])
  print('Z:', obj_range['z'])

  start_time = time.time()
  obj.stretch_to_max()
  elapsed_time = time.time() - start_time

  obj_range = obj.dimension_range()

  print(f"\nStretched non-bounded dimensions using global constants (took {elapsed_time:.2f} s):")
  print(f"\nDimension Range (stretched in {elapsed_time:.2f} s):")
  print('X:', obj_range['x'])
  print('Y:', obj_range['y'])
  print('Z:', obj_range['z'])
  print('-' * 50)

Object: Line in the Sand

Dimension Range:
X: 0.14593999999999996
Y: 0.6710829999999999
Z: 2.0

Stretched non-bounded dimensions using global constants (took 0.02 s):

Dimension Range (stretched in 0.02 s):
X: 1.1967258958852132
Y: 2.0
Z: 2.0
--------------------------------------------------
Object: Rat King

Dimension Range:
X: 0.14158240522736318
Y: 0.5979953838336013
Z: 0.9424218581659534

Stretched non-bounded dimensions using global constants (took 0.04 s):

Dimension Range (stretched in 0.04 s):
X: 1.1609930843997502
Y: 1.7821800994321162
Z: 0.9424218581659534
--------------------------------------------------
Object: Outbreak Perfected

Dimension Range:
X: 0.24389879169790796
Y: 0.5689507899464725
Z: 1.5117135907611965

Stretched non-bounded dimensions using global constants (took 0.03 s):

Dimension Range (stretched in 0.03 s):
X: 2.0
Y: 1.6956197368923742
Z: 1.5117135907611965
--------------------------------------------------
Object: Prometheus Lens

Dimension Range:
X: 0.12

In [ ]:
#@title Verify that dataset dimensional extrema are properly constrained

track_min = {'x': float('inf'), 'y': float('inf'), 'z': float('inf')}
track_max = {'x': float('-inf'), 'y': float('-inf'), 'z': float('-inf')}

for obj in ply_objs:
  max_values = obj.get_max_values()
  min_values = obj.get_min_values()

  for dimension in ['x', 'y', 'z']:
    track_min[dimension] = min(track_min[dimension], min_values[dimension])
    track_max[dimension] = max(track_max[dimension], max_values[dimension])

print("Dimensional Minima:")
print('X:', track_min['x'])
print('Y:', track_min['y'])
print('Z:', track_min['z'])

print("\nDimensional Maxima:")
print('X:', track_max['x'])
print('Y:', track_max['y'])
print('Z:', track_max['z'])

Dimensional Minima:
X: -1.0000000000000002
Y: -1.0
Z: -1.0

Dimensional Maxima:
X: 0.9999999999999997
Y: 1.0
Z: 1.0


In [ ]:
#@title Sort order of vertices and faces numerically to remove arbitrary sample noise {vertical-output: true}

for obj in ply_objs:
    start_time = time.time()

    obj.sort_tri_data()

    elapsed_time = time.time() - start_time

    print(f"Object: {obj.name}\n")

    print(f'Sorted object vertices and faces numerically (took {elapsed_time:.2f} s)')

    print('-' * 50)

Object: Line in the Sand

Sorted object vertices and faces numerically (took 1.65 s)
--------------------------------------------------
Object: Rat King

Sorted object vertices and faces numerically (took 1.24 s)
--------------------------------------------------
Object: Outbreak Perfected

Sorted object vertices and faces numerically (took 1.66 s)
--------------------------------------------------
Object: Prometheus Lens

Sorted object vertices and faces numerically (took 2.69 s)
--------------------------------------------------
Object: Blind Perdition

Sorted object vertices and faces numerically (took 0.71 s)
--------------------------------------------------


In [ ]:
#@title Export normalized PLY files {vertical-output: true}

!rm -r {NORMALIZED_PLY} 2>/dev/null
!mkdir {NORMALIZED_PLY}

for obj in ply_objs:
  start_time = time.time()
  # todo: remove before final model use
  obj.revert_stretch()
  obj.save_file(NORMALIZED_PLY + obj.name + '.ply')
  elapsed_time = time.time() - start_time


  print(f"Object: {obj.name}\n")
  print(f'Exported normalized PLY file (took {elapsed_time:.2f} s)')
  print('-' * 50)

Object: Line in the Sand

Exported normalized PLY file (took 0.74 s)
--------------------------------------------------
Object: Rat King

Exported normalized PLY file (took 0.49 s)
--------------------------------------------------
Object: Outbreak Perfected

Exported normalized PLY file (took 0.36 s)
--------------------------------------------------
Object: Prometheus Lens

Exported normalized PLY file (took 0.39 s)
--------------------------------------------------
Object: Blind Perdition

Exported normalized PLY file (took 0.12 s)
--------------------------------------------------


In [ ]:
!rm -f /content/drive/MyDrive/AutoCalibr/normalized_ply/*
!cp -r /content/intermediates/normalized_ply/ /content/drive/MyDrive/AutoCalibr

# Autoencoder

In [ ]:
#@title Define imports and constants

import ast
import pandas as pd
import os
import time
import warnings
with warnings.catch_warnings():
  warnings.filterwarnings("ignore",category = UserWarning)
  import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import pad_sequences

latent_dim = 50

/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [88]:
#@title Characterize performance for random list generation of a given length and sum {vertical-output: true}

# algorithm specification:
# generate an evenly distributed random list that sums to total_sum, minimum value of any element = 1

# this will be used extensively during padding, so it's worth spending time optimizing

import random
import numpy as np
import pandas as pd


def verify_random_list(lst, length, total_sum):
  if len(lst) != length:
    raise ValueError("Incorrect length")
  elif sum(lst) != total_sum:
    raise ValueError("Incorrect sum")
  elif min(lst) < 1:
    raise ValueError("Minimum value not 1")


def random_list_simple(length, total_sum):
  list = [1]*length

  # Distribute the total_sum across the list
  for i in range(total_sum - length):
    list[random.randint(0, length - 1)] += 1

  return list


def random_list_dist_fill_float(length, total_sum, fill_float):
  # Pre-allocate the list to the target length
  list = [0]*length

  # Range for random value generation for each element
  upper_bound = int(fill_float * ((total_sum - length) / length))

  # Generate initial list and calculate the current sum
  current_sum = 0
  for i in range(length):
    list[i] = random.randint(1, upper_bound)
    current_sum += list[i]

  # If current_sum already exceed total_sum, retry the function
  if current_sum > total_sum:
    return random_list_dist_fill_float(length, total_sum, fill_float)

  # Distribute the remaining sum across the list
  for i in range(current_sum, total_sum):
    list[random.randint(0, length - 1)] += 1

  return list


def random_list_try_dist_then_simple(length, total_sum, fill_float):
  # Pre-allocate the list to the target length
  list = [0]*length

  # Range for random value generation for each element
  upper_bound = int(fill_float * ((total_sum - length) / length))

  # Generate initial list and calculate the current sum
  current_sum = 0
  for i in range(length):
    list[i] = random.randint(1, upper_bound)
    current_sum += list[i]

  # If current_sum already exceed total_sum, retry the function
  if current_sum > total_sum:
    return random_list_simple(length, total_sum)

  # Distribute the remaining sum across the list
  for i in range(current_sum, total_sum):
    list[random.randint(0, length - 1)] += 1

  return list


def random_list_np(length, total_sum):
  lst = [1]*(length-1)
  remainder = total_sum - (length-1)
  lst.append(remainder)
  np.random.shuffle(lst)

  return lst


def random_list_np_multinomal(length, total_sum):
  adjusted_sum = total_sum - length
  result = np.random.multinomial(adjusted_sum, np.ones(length)/length) + 1

  return result.tolist()


scenarios = {
  "Small Scale 10x": (100, 1000),
  "Large Scale 5x": (4000, 21000),
  "Large Scale 2x": (10500, 21000),
  "Large Scale 1.3x": (16000, 21000),
}

functions = [
  random_list_simple,
  random_list_np,
  random_list_np_multinomal
]

float_functions = [
  random_list_dist_fill_float,
  random_list_try_dist_then_simple,
]

num_iterations = 100

float_values = [1.4, 1.6, 1.8]

for scenario_name, scenario_values in scenarios.items():
  print(f"Scenario: {scenario_name} (repeated {num_iterations} iterations)")
  for func in functions:
    start_time = time.time()

    results = []
    errors = set()
    for _ in range(num_iterations):
      try:
        res = func(*scenario_values)
        results.append(res)
        verify_random_list(res, *scenario_values)
      except ValueError as e:
        errors.add(str(e))

    if not errors:
      results_df = pd.DataFrame(results)
      std_dev = results_df.std()
      avg_std_dev = np.mean(std_dev)

    elapsed_time = time.time() - start_time
    error_message = 'Failed' if errors else 'Succeeded'

    print(f"\n{func.__name__} ran in {elapsed_time:.2f} s. {error_message}.")
    if errors:
      print(f"Errors: {', '.join(errors)}")
    else:
      print(f"Avgerage standard deviation across iterations: {avg_std_dev:.2f}")

  # try float functions with different float values
  for float_func in float_functions:
    for float_value in float_values:
      print()
      start_time = time.time()

      results = []
      errors = set()
      for _ in range(num_iterations):
        try:
          res = float_func(*scenario_values, float_value)
          results.append(res)
          verify_random_list(res, *scenario_values)
        except ValueError as e:
          errors.add(str(e))

      if not errors:
        results_df = pd.DataFrame(results)
        std_dev = results_df.std()
        avg_std_dev = np.mean(std_dev)
        avg_std_dev

      elapsed_time = time.time() - start_time
      error_message = 'Failed' if errors else 'Succeeded'

      print(f"{float_func.__name__} (float {float_value}) ran in {elapsed_time:.2f} s. {error_message}.")
      if errors:
        print(f"{', '.join(errors)}")
      else:
        print(f"Avgerage standard deviation across iterations: {avg_std_dev:.2f}")

  print('-' * 50)

Scenario: Small Scale 10x (repeated 100 iterations)

random_list_simple ran in 0.20 s. Succeeded.
Avgerage standard deviation across iterations: 2.99

random_list_np ran in 0.01 s. Succeeded.
Avgerage standard deviation across iterations: 70.56

random_list_np_multinomal ran in 0.01 s. Succeeded.
Avgerage standard deviation across iterations: 2.97

random_list_dist_fill_float (float 1.4) ran in 0.05 s. Succeeded.
Avgerage standard deviation across iterations: 3.92

random_list_dist_fill_float (float 1.6) ran in 0.05 s. Succeeded.
Avgerage standard deviation across iterations: 4.29

random_list_dist_fill_float (float 1.8) ran in 0.04 s. Succeeded.
Avgerage standard deviation across iterations: 4.76

random_list_try_dist_then_simple (float 1.4) ran in 0.06 s. Succeeded.
Avgerage standard deviation across iterations: 3.86

random_list_try_dist_then_simple (float 1.6) ran in 0.04 s. Succeeded.
Avgerage standard deviation across iterations: 4.25

random_list_try_dist_then_simple (float 1.8)

In [ ]:
#@title Import pre-normalized PLY file artifacts(to allow for runtime restart) {vertical-output: true}

import os

ply_objs=[]

for f in os.listdir(NORMALIZED_PLY):
  if f.endswith('.ply'):
    start_time = time.time()
    ply_obj = PlyObject.from_file(filepath=os.path.join(NORMALIZED_PLY, f))
    ply_objs.append(ply_obj)
    elapsed_time = time.time() - start_time

    print(f"Object: {obj.name}\n")
    print(f'Processed pre-normalized PLY file into PlyObject (took {elapsed_time:.2f} s)')
    print(f'\nVertice count: {len(ply_obj.vertices)}')
    print(f'Face count: {len(ply_obj.faces)}')
    print('-' * 50)

In [ ]:
#@title Categorize objects to determine global target padding size and autoencoder input layer size

max_vertices = 0
max_faces = 0

for obj in ply_objs:
  max_vertices = max(max_vertices, len(obj.vertices))
  max_faces = max(max_faces, len(obj.faces))

print(f'Maximum vertices for any object: {max_vertices}')
print(f'Maximum faces for any object: {max_faces}')

max_vertices = int(max_vertices * 1.2)
max_faces = int(max_faces * 1.2)

print(f'\n120% of vertex count: {max_vertices}')
print(f'120% of face count: {max_faces}')

print(f'\nVertex neurons (coordinate count - 3 per vertex): {max_vertices * 3}')
print(f'Face neurons (using encoding - 1 per face): {max_faces}')
print(f'\nTotal Input Neurons: {max_vertices * 3 + max_faces}\n')

Maximum vertices for any object: 16496
Maximum faces for any object: 29689

120% of vertex count: 19795
120% of face count: 35626

Vertex neurons (coordinate count - 3 per vertex): 59385
Face neurons (using encoding - 1 per face): 35626

Total Input Neurons: 95011



In [ ]:
#@title Padding to normalize input neuron count - subdivide faces until either boundary is hit {vertical-output: true}

# note: will only hit one padding bound, not fully padded yet

for obj in ply_objs:
  print(f"Object: {obj.name}\n")
  print(f'Vertex count: {len(obj.vertices)}')
  print(f'Face count: {len(obj.faces)}')

  start_time = time.time()

  obj.subdivide_faces_as_padding(max_faces, max_vertices)

  elapsed_time = time.time() - start_time

  print(f'\nSubdivided faces as padding (took {elapsed_time:.2f} s)')
  print(f'\nVertex count: {len(obj.vertices)} ({max_vertices - len(obj.vertices)} under target)')
  print(f'Face count: {len(obj.faces)} ({max_faces - len(obj.faces)} under target)')
  print('-' * 50)

Object: Outbreak Perfected

Vertex count: 7472
Face count: 28193

Subdivided faces as padding (took 1.90 s)

Vertex count: 14905 (4890 under target)
Face count: 35626 (0 under target)
--------------------------------------------------
Object: Line in the Sand

Vertex count: 16496
Face count: 29689

Subdivided faces as padding (took 0.05 s)

Vertex count: 19795 (0 under target)
Face count: 32988 (2638 under target)
--------------------------------------------------
Object: Rat King

Vertex count: 12219
Face count: 24308

Subdivided faces as padding (took 0.11 s)

Vertex count: 19795 (0 under target)
Face count: 31884 (3742 under target)
--------------------------------------------------
Object: Prometheus Lens

Vertex count: 12590
Face count: 25397

Subdivided faces as padding (took 0.11 s)

Vertex count: 19795 (0 under target)
Face count: 32602 (3024 under target)
--------------------------------------------------
Object: Blind Perdition

Vertex count: 7807
Face count: 14069

Subdivide

In [ ]:
#@title Define variational sampling function

class Sampling(layers.Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
#@title Convert PlyObjects to input arrays {vertical-output: true}

# Doing this step before defining encoder and decoder to avoid system memory cap

vertex_inputs_list = []
face_inputs_list = []
for obj in ply_objs:
  vertex_arr = np.array([[vert.x, vert.y, vert.z] for vert in obj.vertices])

  #vertex_arr = pad_sequences(vertex_arr, maxlen=max_vertices, dtype='float32', padding='post')

  vertex_inputs_list.append(vertex_arr)

  print(f"Converted to array: {obj.name}\n")

  obj = None

ply_objs = None

Converted to array: Ikelos_SG_V1.0.1

Converted to array: Recluse

Converted to array: Tarrabah

Converted to array: Suros Regime

Converted to array: Redrix's Claymore

Converted to array: Telesto

Converted to array: Prospector

Converted to array: Outbreak Perfected

Converted to array: Hereafter

Converted to array: ManannanSR4

Converted to array: Age-Old Bond

Converted to array: Fabian Strategy

Converted to array: Lord of Wolves

Converted to array: Eriana's Vow

Converted to array: Devil's Ruin

Converted to array: Spare Rations

Converted to array: Duality

Converted to array: Izanagis Burden

Converted to array: Bastion

Converted to array: Sunshot

Converted to array: Dust Rock Blues

Converted to array: Optative

Converted to array: Luna's Howl

Converted to array: The Epicurean

Converted to array: Line in the Sand

Converted to array: Patron of Lost Causes

Converted to array: Breachlight

Converted to array: No Time To Explain

Converted to array: Jade Jester

Converted

In [ ]:
#@title Initialize encoder model

vertex_inputs = keras.Input(shape=(max_vertices, 3))
face_inputs = keras.Input(shape=(max_faces, 1))

x = layers.Dense(32, activation="relu")(vertex_inputs)
x = layers.Flatten()(x)

y = layers.Dense(32, activation="relu")(face_inputs)
y = layers.Flatten()(y)

x = layers.Concatenate()([x, y])

x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = keras.Model([vertex_inputs, face_inputs], [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19906, 3)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 38083, 1)]   0           []                               
                                                                                                  
 dense (Dense)                  (None, 19906, 32)    128         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 38083, 32)    64          ['input_2[0][0]']                
                                                                                            

In [ ]:
#@title Initialize decoder model

latent_inputs = keras.Input(shape=(latent_dim,))

x = layers.Dense(512, activation="relu")(latent_inputs)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(2048, activation="relu")(x)
x = layers.Dense(max_vertices * 3, activation="relu")(x)

vertex_outputs = layers.Reshape((max_vertices, 3))(x)

y = layers.Dense(512, activation="relu")(latent_inputs)
y = layers.Dense(1024, activation="relu")(y)
y = layers.Dropout(0.5)(y)
y = layers.Dense(2048, activation="relu")(y)
y = layers.Dense(max_faces, activation="relu")(y)

face_outputs = layers.Reshape((max_faces, 1))(y)

decoder = keras.Model(latent_inputs, [vertex_outputs, face_outputs], name="decoder")
decoder.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 512)          26112       ['input_3[0][0]']                
                                                                                                  
 dense_8 (Dense)                (None, 512)          26112       ['input_3[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 1024)         525312      ['dense_4[0][0]']                
                                                                                            

In [ ]:
#@title Define VAE class

class VAE(keras.Model):
  def __init__(self, encoder, decoder, beta=0.1, **kwargs):
    super().__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.beta = beta
    self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
    self.reconstruction_loss_tracker = keras.metrics.Mean(
      name="reconstruction_loss"
    )
    self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

  @property
  def metrics(self):
    return [
      self.total_loss_tracker,
      self.reconstruction_loss_tracker,
      self.kl_loss_tracker,
    ]

  def train_step(self, data):
    with tf.GradientTape() as tape:
      z_mean, z_log_var, z = self.encoder(data)
      reconstruction = self.decoder(z)
      reconstruction_loss = tf.reduce_mean(
        tf.reduce_sum(
          keras.losses.mse(data, reconstruction), axis=(1, 2)
        )
      )
      kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
      kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
      total_loss = reconstruction_loss + self.beta * kl_loss
    grads = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
    self.total_loss_tracker.update_state(total_loss)
    self.reconstruction_loss_tracker.update_state(reconstruction_loss)
    self.kl_loss_tracker.update_state(kl_loss)
    return {
      "loss": self.total_loss_tracker.result(),
      "reconstruction_loss": self.reconstruction_loss_tracker.result(),
      "kl_loss": self.kl_loss_tracker.result(),
    }

In [ ]:
#@title Train models using defined VAE

# estimating model size = (325mil params * 32 bits) = 1.3GB

'''
vertex_inputs_arr = np.array(vertex_inputs_list)
face_inputs_arr = np.array(face_inputs_list)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit([vertex_inputs_arr, face_inputs_arr], epochs=30, batch_size=128)
'''

'\nvertex_inputs_arr = np.array(vertex_inputs_list)\nface_inputs_arr = np.array(face_inputs_list)\n\nvae = VAE(encoder, decoder)\nvae.compile(optimizer=keras.optimizers.Adam())\nvae.fit([vertex_inputs_arr, face_inputs_arr], epochs=30, batch_size=128)\n'